#### Линеный конгруентный метод

In [52]:
class LinearCongruentialGenerator
{
    private int _a;
    private int _c;
    private int _m;
    private int _currentX;
    private int? _period;
    
    public LinearCongruentialGenerator(int a, int c, int m, int x0, bool generatePeriod = true)
    {
        _a = a;
        _c = c;
        _m = m;
        _currentX = x0;
        _period = generatePeriod ? GeneratePeriod() : null;
    }

    public int Period => _period ?? GeneratePeriod();

    public int Next() {
        _currentX = (_a * _currentX + _c) % _m;
        return _currentX;
    }

    public IEnumerable<int> NextMany(int count) {
        var nums = new List<int>();
        for (var i = 0; i < count; i++) {
            nums.Add(Next());
        }
        return nums;
    }

    private int GeneratePeriod() {
        var x = Next();
        var t = 1;
        while (x != Next())
        {
            t++;
        }
        _period = t;
        return t;
    }
}

In [53]:
var lcg = new LinearCongruentialGenerator(37, 41, 1001, 5);

In [54]:
display(lcg.Period);
display(lcg.NextMany(20))

15

[ 395, 642, 772, 577, 369, 681, 213, 915, 863, 941, 824, 499, 486, 5, 226, 395, 642, 772, 577, 369 ]

In [55]:
#r "nuget:XPlot.Plotly"
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:XPlot.Plotly.Interactive"

Installed Packages Microsoft.Data.Analysis, 0.20.1 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

In [56]:
using XPlot.Plotly;
using Microsoft.Data.Analysis;
using System.Linq;
using XPlot.Plotly.Interactive;

In [57]:
var count = 900;
var nums = lcg.NextMany(count);
var groups = nums.GroupBy(n => n).Select(n => new { X = n.Key, Count = n.Count() });

In [58]:
var bar = new Bar
{   
    x = groups.Select(x => x.X),
    y = groups.Select(x => x.Count),
    marker = new Marker { color = "green" }
};

var layout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "Значение",
    },
    yaxis = new Yaxis
    {
        title = "Количество"
    },
    title = $"Количество одинаковых значений для сгенерированных {count} значений"
};

Chart.Plot(bar, layout)

<!DOCTYPE html>

Проверим посследовательность из младших разрядов

In [59]:
var littles = nums.Select(x => x % 10);
var groupsLittle = littles.GroupBy(n => n).Select(n => new { X = n.Key, Count = n.Count() });
display(littles)

[ 1, 3, 5, 3, 1, 4, 9, 6, 5, 6, 5, 2, 2, 7, 9, 1, 3, 5, 3, 1 ... (more) ]

In [60]:
var barLittle = new Bar
{   
    x = groupsLittle.Select(x => x.X),
    y = groupsLittle.Select(x => x.Count),
    marker = new Marker { color = "green" }
};

var layoutLittle = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "Значение",
    },
    yaxis = new Yaxis
    {
        title = "Количество"
    },
    title = $"Младшие разряды"
};

Chart.Plot(barLittle, layoutLittle)

<!DOCTYPE html>

#### ЦПТ

In [61]:
class CLT
{
    private LinearCongruentialGenerator _lcg;
    public int Mean => 6;
    public int Variance => 1;
    public int LcgPeriod { get; init;}

    public CLT()
    {
        _lcg = new LinearCongruentialGenerator(211, 1663, 7875, 3, true);
        LcgPeriod = _lcg.Period;
    }

    public double Next()
    {
        return (_lcg.NextMany(12).Select(x => x ).Sum() - Mean) / Variance;
    }

    public IEnumerable<double> NextMany(int count) {
        var nums = new List<double>();
        for (var i = 0; i < count; i++) {
            nums.Add(Next());
        }
        return nums;
    }
}

In [62]:
var clt = new CLT();
display(clt.LcgPeriod);

7875

In [63]:
var count = clt.LcgPeriod;
var nums = clt.NextMany(count);
var ranges = new[] { 0, 5000, 10000, 15000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000 };
var groups = nums.GroupBy(n => ranges.FirstOrDefault(r => r >= n)).Select(n => new { X = n.Key, Count = n.Count() });

var normalNums = new List<double>();
var normals = new List<double>();
var s = 10000;
var coef = 47000000;
var m = 50000;
for (double i = m - 3 * s; i < m + 3 * s; i += s / 100)
{
    var n =  coef * (1 / (s * Math.Sqrt(2 * Math.PI)) * Math.Exp(-0.5 * Math.Pow((i - m) / s, 2)));
    normalNums.Add(n);
    normals.Add(i);
}

In [64]:
var scatter = new Scatter
{   
    x = normals,
    y = normalNums,
    marker = new Marker { color = "red" }
};

var bar = new Bar
{   
    x = groups.Select(x => x.X),
    y = groups.Select(x => x.Count),
    marker = new Marker { color = "green" }
};

var layout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "Значение",
    },
    yaxis = new Yaxis
    {
        title = "Количество"
    },
    title = $"Центральная предельная теорема"
};

Chart.Plot(new List<Trace>() { bar, scatter }, layout)

<!DOCTYPE html>

#### Бокс-Мюллер

In [65]:
class BMT
{
    private LinearCongruentialGenerator _lcg;
    public int Mean => 6;
    public int Variance => 1;

    public int LcgPeriod { get; init;}
    private int _max;

    public BMT()
    {
        _lcg = new LinearCongruentialGenerator(211, 1663, 7875, 3, true);
        LcgPeriod = _lcg.Period;
        _max = _lcg.NextMany(LcgPeriod).Max();
    }

    public (double, double, double, double) Next()
    {
        var x = _lcg.Next() / (_max * 1.0);
        var y = _lcg.Next() / (_max * 1.0);
        var s = x * x + y * y;

        if (s == 0 || s > 1) 
        {
            return Next();
        }

        var z1 = x * Math.Sqrt(-2 * Math.Log(s) / s);
        var z2 = y * Math.Sqrt(-2 * Math.Log(s) / s);

        return (z1, z2, x, y);
    }

    public IEnumerable<(double, double, double, double)> NextMany(int count) {
        var nums = new List<(double, double, double, double)>();
        for (var i = 0; i < count; i++) {
            nums.Add(Next());
        }
        return nums;
    }
}

In [66]:
var bmt = new BMT();
display(bmt.LcgPeriod)

7875

In [67]:
var count = Convert.ToInt32(500);
var dNums = bmt.NextMany(count);


In [68]:
var scatterNew = new Scatter
{   
    x = dNums.Select(x => x.Item1),
    y = dNums.Select(x => x.Item2),
    mode = "markers",
    marker = new Marker 
    { 
        color = "blue",
        size = 6,
        line = new Line {
            color = "white",
            width = 0.5
        }
    }
};

var scatterOld = new Scatter
{   
    x = dNums.Select(x => x.Item3),
    y = dNums.Select(x => x.Item4),
    mode = "markers",
    marker = new Marker 
    { 
        color = "green",
        size = 6,
        line = new Line {
            color = "white",
            width = 0.5
        }
    }
};

var layout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "x",
    },
    yaxis = new Yaxis
    {
        title = "y"
    },
    title = $"Бокс-Мюллер",
    
};

var plot = Chart.Plot(new List<Trace>() { scatterOld, scatterNew }, layout);
plot.Height = 700;
plot.Width = 700;
plot

<!DOCTYPE html>